# EDA Using Athena

**We have resolved the column names, datatypes differences of the parquet files and created a new data catalog table**

- Parquet files are usually 10% to 50% in size v/s csv files for the same dataset
- It's not possible to load this huge data in pandas dataframe.
- So we use Athena to explore the dataset.

Docs References: https://docs.aws.amazon.com/athena/latest/ug/what-is.html

In [1]:
!pip install awswrangler

In [15]:
import pandas as pd
import boto3
import awswrangler as wr
import sagemaker

In [37]:
boto_session = boto3.session.Session()
s3_client = boto_session.client('s3')
s3_output=""

## Change the location of s3 data if you have moved it.

In [17]:
#query = """
#    ALTER TABLE raw_data
#    SET LOCATION
#"""
#df = wr.athena.start_query_execution(sql=query, database='nyc_taxi_data')
#df

### Get sample of 10 rows

In [46]:
query = "select * from schema_corrected_data limit 100"
df = wr.athena.read_sql_query(sql=query, database='nyc_taxi_data')
wr.s3.to_csv(df=df, path=f"{s3_output}/sample_100_rows.csv", index=False)
df

[07/03/25 10:54:39] INFO     Created CTAS table                                                       ]8;id=955184;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=292179;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_40703507253d451d822385c8fae296a4"                         

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,cbd_congestion_fee,tpep_pickup_date
0,1,2024-01-03 00:03:28,2024-01-03 00:22:43,<NA>,0.00,<NA>,<NA>,193,92,0,...,0.00,0.5,0.00,0.00,1.0,28.46,NaN,NaN,NaN,2024-01-03
1,2,2019-09-25 08:53:41,2019-09-25 16:20:56,1,2.32,1,N,162,246,1,...,0.00,0.5,3.96,0.00,0.3,23.76,2.5,NaN,NaN,2019-09-25
2,2,2024-01-03 00:01:18,2024-01-03 00:29:13,<NA>,9.85,<NA>,<NA>,137,62,0,...,0.00,0.5,2.00,0.00,1.0,42.44,NaN,NaN,NaN,2024-01-03
3,1,2024-01-03 00:45:32,2024-01-03 01:26:55,<NA>,19.30,<NA>,<NA>,132,143,0,...,1.75,0.5,4.13,6.94,1.0,86.82,NaN,NaN,NaN,2024-01-03
4,2,2024-01-03 00:29:00,2024-01-03 00:44:00,<NA>,2.88,<NA>,<NA>,137,50,0,...,0.00,0.5,3.00,0.00,1.0,23.67,NaN,NaN,NaN,2024-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,2024-01-03 04:32:31,2024-01-03 04:58:58,<NA>,18.39,<NA>,<NA>,181,132,0,...,0.00,0.5,11.52,0.00,1.0,69.11,NaN,NaN,NaN,2024-01-03
96,2,2024-01-03 04:47:00,2024-01-03 05:02:00,<NA>,8.31,<NA>,<NA>,236,138,0,...,0.00,0.5,8.54,6.94,1.0,51.26,NaN,NaN,NaN,2024-01-03
97,2,2024-01-03 04:58:42,2024-01-03 05:26:33,<NA>,14.17,<NA>,<NA>,89,132,0,...,0.00,0.5,13.72,0.00,1.0,68.59,NaN,NaN,NaN,2024-01-03
98,1,2024-01-03 04:25:21,2024-01-03 04:42:36,<NA>,0.00,<NA>,<NA>,79,138,0,...,7.00,0.5,7.20,0.00,1.0,57.20,NaN,NaN,NaN,2024-01-03


In [54]:
query = """
    SELECT 
    column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'nyc_taxi_data' 
    AND table_name = 'schema_corrected_data';
    """ 

table_schema = wr.athena.read_sql_query(sql=query, database='nyc_taxi_data')
columns = table_schema['column_name'].tolist()
datatypes = table_schema['data_type'].tolist()
table_schema

[07/03/25 11:12:38] INFO     Created CTAS table                                                       ]8;id=109688;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=841870;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_60cf73420fae41219335ee529b5a8b2a"                         

,column_name,data_type
0,vendorid,integer
1,tpep_pickup_datetime,timestamp(3)
2,tpep_dropoff_datetime,timestamp(3)
3,passenger_count,integer
4,trip_distance,double
5,ratecodeid,integer
6,store_and_fwd_flag,varchar
7,pulocationid,integer
8,dolocationid,integer
9,payment_type,integer


### Get By Year Summary

In [39]:
query = """
    SELECT 
        year(tpep_pickup_datetime) AS year, 
        COUNT(*) as total_trips, 
        ROUND(SUM(total_amount) / 1000000, 0) as total_amount,
        ROUND(SUM(fare_amount) / 1000000, 0) as fare_amount, 
        ROUND(SUM(tip_amount) / 1000000, 0) as tip_amount
    FROM schema_corrected_data 
    GROUP BY year(tpep_pickup_datetime)
    ORDER BY 1
"""
df = wr.athena.read_sql_query(sql=query, database='nyc_taxi_data')
wr.s3.to_csv(df=df, path=f"{s3_output}/summary_by_year.csv", index=False)
df

[07/03/25 10:42:12] INFO     Created CTAS table                                                       ]8;id=735724;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=601173;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_2f5751eae3214ab284a763c96c3b8080"                         

,year,total_trips,total_amount,fare_amount,tip_amount
0,2001,15,0.0,0.0,0.0
1,2002,478,0.0,0.0,0.0
2,2003,33,0.0,0.0,0.0
3,2004,1,0.0,0.0,0.0
4,2007,1,0.0,0.0,0.0
5,2008,366,0.0,0.0,0.0
6,2009,744,0.0,0.0,0.0
7,2010,1,0.0,0.0,0.0
8,2011,4,0.0,0.0,0.0
9,2012,1,0.0,0.0,0.0


### Missingness in the data

In [50]:
#        date_format(tpep_pickup_datetime, "%Y%m) as month_year,
query = f"""
    SELECT 
        year(tpep_pickup_datetime) as year,
        month(tpep_pickup_datetime) as month,
        COUNT(*) as total_rows,
        {' , '.join([
            'ROUND(100 - (100 * COUNT({}) / COUNT(*)), 1) AS {}'.format(col, col) 
            for col in columns
        ])}
    FROM schema_corrected_data
    GROUP BY month(tpep_pickup_datetime), year(tpep_pickup_datetime) 
    ORDER BY year, month
"""
print(query)
df = wr.athena.read_sql_query(sql=query, database='nyc_taxi_data', s3_output="s3://sagemaker-us-east-1-205930620783/NYC_Taxi_Prediction/")
wr.s3.to_csv(df=df, path=f"{s3_output}/missingness_by_month_year.csv", index=False)#df.to_csv("")


    SELECT 
        year(tpep_pickup_datetime) as year,
        month(tpep_pickup_datetime) as month,
        COUNT(*) as total_rows,
        ROUND(100 - (100 * COUNT(vendorid) / COUNT(*)), 1) AS vendorid , ROUND(100 - (100 * COUNT(tpep_pickup_datetime) / COUNT(*)), 1) AS tpep_pickup_datetime , ROUND(100 - (100 * COUNT(tpep_dropoff_datetime) / COUNT(*)), 1) AS tpep_dropoff_datetime , ROUND(100 - (100 * COUNT(passenger_count) / COUNT(*)), 1) AS passenger_count , ROUND(100 - (100 * COUNT(trip_distance) / COUNT(*)), 1) AS trip_distance , ROUND(100 - (100 * COUNT(ratecodeid) / COUNT(*)), 1) AS ratecodeid , ROUND(100 - (100 * COUNT(store_and_fwd_flag) / COUNT(*)), 1) AS store_and_fwd_flag , ROUND(100 - (100 * COUNT(pulocationid) / COUNT(*)), 1) AS pulocationid , ROUND(100 - (100 * COUNT(dolocationid) / COUNT(*)), 1) AS dolocationid , ROUND(100 - (100 * COUNT(payment_type) / COUNT(*)), 1) AS payment_type , ROUND(100 - (100 * COUNT(fare_amount) / COUNT(*)), 1) AS fare_amount , ROUND(100 - (1

[07/03/25 10:58:44] INFO     Created CTAS table                                                       ]8;id=569330;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=157501;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_225cd5298264459a8ce1754cfdc608a9"                         

{'paths': ['s3://sagemaker-us-east-1-205930620783/NYC_Taxi_Prediction/athena_query_results/missingness_by_month_year.csv'],
 'partitions_values': {}}

## Distribution of each columns

In [66]:
numeric_types = ['integer', 'bigint', 'double', 'float', 'decimal']
categorical_types = ['varchar', 'string', 'char', 'boolean']

numeric_f, categorical_f = [], []
for col_name, data_type in zip(columns, datatypes):
    print(col_name)
    if data_type in numeric_types:
        numeric_f.append(col_name)
    elif data_type in categorical_types:
        categorical_f.append(col_name)
    else:
        print("pass")

print("Numerical Columns: ", numeric_f)
print("Categorical Columns: ", categorical_f)


vendorid
tpep_pickup_datetime
pass
tpep_dropoff_datetime
pass
passenger_count
trip_distance
ratecodeid
store_and_fwd_flag
pulocationid
dolocationid
payment_type
fare_amount
extra
mta_tax
tip_amount
tolls_amount
improvement_surcharge
total_amount
congestion_surcharge
airport_fee
cbd_congestion_fee
tpep_pickup_date
Numerical Columns:  ['vendorid', 'passenger_count', 'trip_distance', 'ratecodeid', 'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'cbd_congestion_fee']
Categorical Columns:  ['store_and_fwd_flag', 'tpep_pickup_date']


### Distribution of Numerical Fields


In [76]:
summary_expr = []
for col in numeric_f:
    summary_expr.extend([
        f"ROUND(MIN({col}),3) AS {col}_min",
        f"ROUND(MAX({col}),3) AS {col}_max",
        f"ROUND(AVG({col}),3) AS {col}_mean",
        f"ROUND(STDDEV({col}),3) AS {col}_std",
        f"ROUND(approx_percentile({col}, 0.25),3) AS {col}_p25",
        f"ROUND(approx_percentile({col}, 0.5),3) AS {col}_p50",
        f"ROUND(approx_percentile({col}, 0.75),3) AS {col}_p75"
    ])

query = f"""
    SELECT
        year(tpep_pickup_datetime) as year,
        {',\n '.join(summary_expr)}
    FROM schema_corrected_data
    GROUP BY year(tpep_pickup_datetime)
    ORDER BY year
"""
print(query)
df = wr.athena.read_sql_query(sql=query, database='nyc_taxi_data', s3_output="s3://sagemaker-us-east-1-205930620783/NYC_Taxi_Prediction/")
wr.s3.to_csv(df=df, path=f"{s3_output}/numeric_features_summary.csv", index=False)#df.to_csv("")

df.head(10)


    SELECT
        year(tpep_pickup_datetime) as year,
        ROUND(MIN(vendorid),3) AS vendorid_min,
 ROUND(MAX(vendorid),3) AS vendorid_max,
 ROUND(AVG(vendorid),3) AS vendorid_mean,
 ROUND(STDDEV(vendorid),3) AS vendorid_std,
 ROUND(approx_percentile(vendorid, 0.25),3) AS vendorid_p25,
 ROUND(approx_percentile(vendorid, 0.5),3) AS vendorid_p50,
 ROUND(approx_percentile(vendorid, 0.75),3) AS vendorid_p75,
 ROUND(MIN(passenger_count),3) AS passenger_count_min,
 ROUND(MAX(passenger_count),3) AS passenger_count_max,
 ROUND(AVG(passenger_count),3) AS passenger_count_mean,
 ROUND(STDDEV(passenger_count),3) AS passenger_count_std,
 ROUND(approx_percentile(passenger_count, 0.25),3) AS passenger_count_p25,
 ROUND(approx_percentile(passenger_count, 0.5),3) AS passenger_count_p50,
 ROUND(approx_percentile(passenger_count, 0.75),3) AS passenger_count_p75,
 ROUND(MIN(trip_distance),3) AS trip_distance_min,
 ROUND(MAX(trip_distance),3) AS trip_distance_max,
 ROUND(AVG(trip_distance),3) AS trip_

[07/03/25 11:37:31] INFO     Created CTAS table                                                       ]8;id=303524;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=722334;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_4bec43adeccd42eb923a4561a29c77d5"                         

,year,vendorid_min,vendorid_max,vendorid_mean,vendorid_std,vendorid_p25,vendorid_p50,vendorid_p75,passenger_count_min,passenger_count_max,...,airport_fee_p25,airport_fee_p50,airport_fee_p75,cbd_congestion_fee_min,cbd_congestion_fee_max,cbd_congestion_fee_mean,cbd_congestion_fee_std,cbd_congestion_fee_p25,cbd_congestion_fee_p50,cbd_congestion_fee_p75
0,2001,2,2,2.0,0.0,2,2,2,1,2,...,0.0,0.0,1.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,2,2,2.0,0.0,2,2,2,1,4,...,0.0,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,2,2,2.0,0.0,2,2,2,1,6,...,0.0,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,2,2,2.0,NaN,2,2,2,1,1,...,0.0,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007,2,2,2.0,NaN,2,2,2,1,1,...,0.0,0.0,0.000,0.75,0.75,0.750,NaN,0.75,0.75,0.75
5,2008,2,2,2.0,0.0,2,2,2,1,6,...,0.0,0.0,0.313,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2009,2,2,2.0,0.0,2,2,2,1,6,...,0.0,0.0,0.000,0.00,0.75,0.375,0.53,0.00,0.75,0.75
7,2010,2,2,2.0,NaN,2,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2011,2,2,2.0,0.0,2,2,2,1,1,...,0.0,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,2,2,2.0,NaN,2,2,2,1,1,...,0.0,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
for col in categorical_f:
    top_cat_sql = f"""
        SELECT {col}, COUNT(*) AS count 
        FROM schema_corrected_data
        GROUP BY {col}
        ORDER BY count DESC
        LIMIT 5
    """
    df = wr.athena.read_sql_query(top_cat_sql, database='nyc_taxi_data')
    display(df)

[07/03/25 11:39:26] INFO     Created CTAS table                                                       ]8;id=198698;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=914422;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_05dcfb71b4524e2cbd545711ba617bf2"                         

,store_and_fwd_flag,count
0,N,255934077
1,<NA>,13660868
2,Y,2239476


[07/03/25 11:39:36] INFO     Created CTAS table                                                       ]8;id=330361;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py\_utils.py]8;;\:]8;id=69807;file:///opt/conda/lib/python3.12/site-packages/awswrangler/athena/_utils.py#872\872]8;;\
                             "nyc_taxi_data"."temp_table_be166a097ae04d358e06745fa2d65849"                         

,tpep_pickup_date,count
0,2019-02-01,299261
1,2019-04-05,293969
2,2019-01-25,292499
3,2019-03-07,292405
4,2019-01-11,291714
